In [1]:
import torch

In [29]:
w2 = torch.tensor([
    [1, 0.1]
]).float()

tensor([[ 2.4403e+07, -2.4403e+08],
        [-2.4403e+08,  2.4403e+09]])

In [22]:
n = 100
x = torch.randn(100, 2)
y = x[:,0].clone()

In [32]:
ls = torch.linalg.inv(x.T @ x) @ x.T @ y

In [39]:
ls

tensor([ 1.0000e+00, -7.4506e-09])

In [42]:
ls.unsqueeze(0)

tensor([[ 1.0000e+00, -7.4506e-09]])

In [43]:
w2.T

tensor([[1.0000],
        [0.1000]])

In [46]:
w1 = torch.outer(ls, w2.view(2)) / torch.norm(w2)

In [47]:
w1

tensor([[ 9.9504e-01,  9.9504e-02],
        [-7.4136e-09, -7.4136e-10]])